In [4]:
import json
import requests
import time
import logging
import io


In [11]:
class Vacancy():
    def __init__(self, area=113):
        self.area = area
        
    def get_page(self, work_name, page=0):
        
        params = {
            'text': work_name,
            'area': self.area,
            'page': page,
            'per_page': 100  # default 100 (100 vacancy per each page)
        }
        
        try:
            req = requests.get('https://api.hh.ru/vacancies', params)
            print(req.url)
            list_of_vacancies = req.content.decode()
            req.close()
        except requests.exceptions.RequestException as e:  # This is the correct syntax
            raise SystemExit(e)

        return list_of_vacancies

    def get_vacancy():
        skills = []
        

In [29]:
vacancy = Vacancy()
vacancy.get_page('', 1)

https://api.hh.ru/vacancies?text=&area=113&page=1&per_page=100


'{"items":[{"id":"44489318","premium":false,"name":"Менеджер по продажам","department":null,"has_test":false,"response_letter_required":false,"area":{"id":"2","name":"Санкт-Петербург","url":"https://api.hh.ru/areas/2"},"salary":{"from":null,"to":100000,"currency":"RUR","gross":false},"type":{"id":"open","name":"Открытая"},"address":{"city":"Санкт-Петербург","street":"Социалистическая улица","building":"14","description":null,"lat":59.922625,"lng":30.34306,"raw":"Санкт-Петербург, Социалистическая улица, 14","metro":{"station_name":"Владимирская","line_name":"Кировско-Выборгская","station_id":"14.200","line_id":"14","lat":59.927628,"lng":30.347898},"metro_stations":[{"station_name":"Владимирская","line_name":"Кировско-Выборгская","station_id":"14.200","line_id":"14","lat":59.927628,"lng":30.347898}],"id":"1088878"},"response_url":null,"sort_point_distance":null,"published_at":"2021-05-20T16:40:38+0300","created_at":"2021-05-20T16:40:38+0300","archived":false,"apply_alternate_url":"https: